In [4]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from glob import glob

In [5]:
img_dir = '/data/public/rw/minjee/handhygiene/images/'
excel_path = os.path.join(os.getcwd(), 'hh_video_data.xlsx')
label_path = os.path.join(os.getcwd(), 'handhygiene_label.csv') 

In [19]:
hh_dict = {'video_id':[], 'date':[], 'img_name':[]}
def create_hh_csv(img_dir):
    for image in tqdm(glob(img_dir+'*.jpg')):
        img_name= image.split('/')[-1].split('.')[0]

        video_id = img_name.split('_')[0]
        date = img_name.split('_')[1]
        frame_num = img_name[-6:]
        
        hh_dict['video_id'].append(video_id)
        hh_dict['date'].append(date)
        hh_dict['img_name'].append(img_name+'.jpg')

In [20]:
create_hh_csv(img_dir)

100%|██████████| 4122/4122 [00:00<00:00, 350070.28it/s]


In [21]:
df = pd.DataFrame(hh_dict)
df.sort_values('img_name', inplace=True)
df.reset_index(inplace=True)
df.drop('index', 1, inplace=True)

In [22]:
df

,date,img_name,video_id
0,20180806,1_20180806_frames000000.jpg,1
1,20180806,1_20180806_frames000001.jpg,1
2,20180806,1_20180806_frames000002.jpg,1
3,20180806,1_20180806_frames000003.jpg,1
4,20180806,1_20180806_frames000004.jpg,1
5,20180806,1_20180806_frames000005.jpg,1
6,20180806,1_20180806_frames000006.jpg,1
7,20180806,1_20180806_frames000007.jpg,1
8,20180806,1_20180806_frames000008.jpg,1
9,20180806,1_20180806_frames000009.jpg,1


In [25]:
df_excel = pd.read_excel(excel_path)

In [35]:
hh_dict = {'date':[], 'img_name':[], 'video_id':[], 'target':[] }
temp_array = []
for row in df.values[:]:
    video_id = int(row[2])
    frame_num = row[1].split('_')[-1].split('.')[0][-6:]
    for vid, target, length in df_excel[['video_id', 'target_frame', 'frame_length']].values:
        if video_id is not vid:
            continue
        if target is np.nan:
            continue
        
        target_frames = target.strip().split(',')
        target_frames = list(map(lambda x: "%.6d" % int(x) ,target_frames))
        if frame_num in target_frames:
            # update target frame list
            temp_array = list(map(lambda n: "%.6d" % (n + int(frame_num)), range(int(length))))
            row = np.append(row, [1])
            continue
        if frame_num in temp_array:
            row = np.append(row, [1])
        else:
            row = np.append(row, [0])
        print(row)   
        hh_dict['date'].append(row[0]) 
        hh_dict['img_name'].append(row[1])
        hh_dict['video_id'].append(row[2])
        hh_dict['target'].append(row[3])

['20180806' '1_20180806_frames000000.jpg' '1' 0]
['20180806' '1_20180806_frames000001.jpg' '1' 0]
['20180806' '1_20180806_frames000002.jpg' '1' 0]
['20180806' '1_20180806_frames000003.jpg' '1' 0]
['20180806' '1_20180806_frames000004.jpg' '1' 0]
['20180806' '1_20180806_frames000005.jpg' '1' 0]
['20180806' '1_20180806_frames000006.jpg' '1' 0]
['20180806' '1_20180806_frames000007.jpg' '1' 0]
['20180806' '1_20180806_frames000008.jpg' '1' 0]
['20180806' '1_20180806_frames000009.jpg' '1' 0]
['20180806' '1_20180806_frames000010.jpg' '1' 0]
['20180806' '1_20180806_frames000011.jpg' '1' 0]
['20180806' '1_20180806_frames000012.jpg' '1' 0]
['20180806' '1_20180806_frames000013.jpg' '1' 0]
['20180806' '1_20180806_frames000014.jpg' '1' 0]
['20180806' '1_20180806_frames000015.jpg' '1' 0]
['20180806' '1_20180806_frames000016.jpg' '1' 0]
['20180806' '1_20180806_frames000017.jpg' '1' 0]
['20180806' '1_20180806_frames000018.jpg' '1' 0]
['20180806' '1_20180806_frames000019.jpg' '1' 0]
['20180806' '1_20180

['20180806' '1_20180806_frames000170.jpg' '1' 0]
['20180806' '1_20180806_frames000171.jpg' '1' 0]
['20180806' '1_20180806_frames000172.jpg' '1' 0]
['20180806' '1_20180806_frames000173.jpg' '1' 0]
['20180806' '1_20180806_frames000174.jpg' '1' 0]
['20180806' '1_20180806_frames000175.jpg' '1' 0]
['20180806' '1_20180806_frames000176.jpg' '1' 0]
['20180806' '1_20180806_frames000177.jpg' '1' 0]
['20180806' '1_20180806_frames000178.jpg' '1' 0]
['20180806' '1_20180806_frames000179.jpg' '1' 0]
['20180806' '1_20180806_frames000180.jpg' '1' 0]
['20180806' '1_20180806_frames000181.jpg' '1' 0]
['20180806' '1_20180806_frames000182.jpg' '1' 0]
['20180806' '1_20180806_frames000183.jpg' '1' 0]
['20180806' '1_20180806_frames000184.jpg' '1' 0]
['20180806' '1_20180806_frames000185.jpg' '1' 0]
['20180806' '1_20180806_frames000186.jpg' '1' 0]
['20180806' '1_20180806_frames000187.jpg' '1' 0]
['20180806' '1_20180806_frames000188.jpg' '1' 0]
['20180806' '1_20180806_frames000189.jpg' '1' 0]
['20180806' '1_20180

['20180806' '1_20180806_frames000341.jpg' '1' 0]
['20180806' '1_20180806_frames000342.jpg' '1' 0]
['20180806' '1_20180806_frames000343.jpg' '1' 0]
['20180806' '1_20180806_frames000344.jpg' '1' 0]
['20180806' '1_20180806_frames000345.jpg' '1' 0]
['20180806' '1_20180806_frames000346.jpg' '1' 0]
['20180806' '1_20180806_frames000347.jpg' '1' 0]
['20180806' '1_20180806_frames000348.jpg' '1' 0]
['20180806' '1_20180806_frames000349.jpg' '1' 0]
['20180806' '1_20180806_frames000350.jpg' '1' 0]
['20180806' '1_20180806_frames000351.jpg' '1' 0]
['20180806' '1_20180806_frames000352.jpg' '1' 0]
['20180806' '1_20180806_frames000353.jpg' '1' 0]
['20180806' '1_20180806_frames000354.jpg' '1' 0]
['20180806' '1_20180806_frames000355.jpg' '1' 0]
['20180806' '1_20180806_frames000356.jpg' '1' 0]
['20180806' '1_20180806_frames000357.jpg' '1' 0]
['20180806' '1_20180806_frames000358.jpg' '1' 0]
['20180806' '1_20180806_frames000359.jpg' '1' 0]
['20180806' '1_20180806_frames000360.jpg' '1' 0]
['20180806' '1_20180

['20180806' '1_20180806_frames000512.jpg' '1' 0]
['20180806' '1_20180806_frames000513.jpg' '1' 0]
['20180806' '1_20180806_frames000514.jpg' '1' 0]
['20180806' '1_20180806_frames000515.jpg' '1' 0]
['20180806' '1_20180806_frames000516.jpg' '1' 0]
['20180806' '1_20180806_frames000517.jpg' '1' 0]
['20180806' '1_20180806_frames000518.jpg' '1' 0]
['20180806' '1_20180806_frames000519.jpg' '1' 0]
['20180806' '1_20180806_frames000520.jpg' '1' 0]
['20180806' '1_20180806_frames000521.jpg' '1' 0]
['20180806' '1_20180806_frames000522.jpg' '1' 0]
['20180806' '1_20180806_frames000523.jpg' '1' 0]
['20180806' '1_20180806_frames000524.jpg' '1' 0]
['20180806' '1_20180806_frames000525.jpg' '1' 0]
['20180806' '1_20180806_frames000526.jpg' '1' 0]
['20180806' '1_20180806_frames000527.jpg' '1' 0]
['20180806' '1_20180806_frames000528.jpg' '1' 0]
['20180806' '1_20180806_frames000529.jpg' '1' 0]
['20180806' '1_20180806_frames000530.jpg' '1' 0]
['20180806' '1_20180806_frames000531.jpg' '1' 0]
['20180806' '1_20180

['20180806' '1_20180806_frames000683.jpg' '1' 0]
['20180806' '1_20180806_frames000684.jpg' '1' 0]
['20180806' '1_20180806_frames000685.jpg' '1' 0]
['20180806' '1_20180806_frames000686.jpg' '1' 0]
['20180806' '1_20180806_frames000687.jpg' '1' 0]
['20180806' '1_20180806_frames000688.jpg' '1' 0]
['20180806' '1_20180806_frames000689.jpg' '1' 0]
['20180806' '1_20180806_frames000690.jpg' '1' 0]
['20180806' '1_20180806_frames000691.jpg' '1' 0]
['20180806' '1_20180806_frames000692.jpg' '1' 0]
['20180806' '1_20180806_frames000693.jpg' '1' 0]
['20180806' '1_20180806_frames000694.jpg' '1' 0]
['20180806' '1_20180806_frames000695.jpg' '1' 0]
['20180806' '1_20180806_frames000696.jpg' '1' 0]
['20180806' '1_20180806_frames000697.jpg' '1' 0]
['20180806' '1_20180806_frames000698.jpg' '1' 0]
['20180806' '1_20180806_frames000699.jpg' '1' 0]
['20180806' '1_20180806_frames000700.jpg' '1' 0]
['20180806' '1_20180806_frames000701.jpg' '1' 0]
['20180806' '1_20180806_frames000702.jpg' '1' 0]
['20180806' '1_20180

...

['20180920' '6_20180920_frames000420.jpg' '6' 0]
['20180920' '6_20180920_frames000421.jpg' '6' 0]
['20180920' '6_20180920_frames000422.jpg' '6' 0]
['20180920' '6_20180920_frames000423.jpg' '6' 0]
['20180920' '6_20180920_frames000424.jpg' '6' 0]
['20180920' '6_20180920_frames000425.jpg' '6' 0]
['20180920' '6_20180920_frames000426.jpg' '6' 0]
['20180920' '6_20180920_frames000427.jpg' '6' 0]
['20180920' '6_20180920_frames000428.jpg' '6' 0]
['20180920' '6_20180920_frames000429.jpg' '6' 0]
['20180920' '6_20180920_frames000430.jpg' '6' 0]
['20180920' '6_20180920_frames000431.jpg' '6' 0]
['20180920' '6_20180920_frames000432.jpg' '6' 0]
['20180920' '6_20180920_frames000433.jpg' '6' 0]
['20180920' '6_20180920_frames000434.jpg' '6' 0]
['20180920' '6_20180920_frames000435.jpg' '6' 0]
['20180920' '6_20180920_frames000436.jpg' '6' 0]
['20180920' '6_20180920_frames000437.jpg' '6' 0]
['20180920' '6_20180920_frames000438.jpg' '6' 0]
['20180920' '6_20180920_frames000439.jpg' '6' 0]
['20180920' '6_20180

['20180920' '7_20180920_frames000021.jpg' '7' 0]
['20180920' '7_20180920_frames000022.jpg' '7' 0]
['20180920' '7_20180920_frames000023.jpg' '7' 0]
['20180920' '7_20180920_frames000024.jpg' '7' 0]
['20180920' '7_20180920_frames000025.jpg' '7' 0]
['20180920' '7_20180920_frames000026.jpg' '7' 0]
['20180920' '7_20180920_frames000027.jpg' '7' 0]
['20180920' '7_20180920_frames000028.jpg' '7' 0]
['20180920' '7_20180920_frames000029.jpg' '7' 0]
['20180920' '7_20180920_frames000030.jpg' '7' 0]
['20180920' '7_20180920_frames000031.jpg' '7' 0]
['20180920' '7_20180920_frames000032.jpg' '7' 0]
['20180920' '7_20180920_frames000033.jpg' '7' 0]
['20180920' '7_20180920_frames000034.jpg' '7' 0]
['20180920' '7_20180920_frames000035.jpg' '7' 0]
['20180920' '7_20180920_frames000036.jpg' '7' 0]
['20180920' '7_20180920_frames000037.jpg' '7' 0]
['20180920' '7_20180920_frames000038.jpg' '7' 0]
['20180920' '7_20180920_frames000039.jpg' '7' 0]
['20180920' '7_20180920_frames000040.jpg' '7' 0]
['20180920' '7_20180

['20180920' '7_20180920_frames000194.jpg' '7' 0]
['20180920' '7_20180920_frames000195.jpg' '7' 0]
['20180920' '7_20180920_frames000196.jpg' '7' 0]
['20180920' '7_20180920_frames000197.jpg' '7' 0]
['20180920' '7_20180920_frames000198.jpg' '7' 0]
['20180920' '7_20180920_frames000199.jpg' '7' 0]
['20180920' '7_20180920_frames000201.jpg' '7' 1]
['20180920' '7_20180920_frames000202.jpg' '7' 1]
['20180920' '7_20180920_frames000203.jpg' '7' 1]
['20180920' '7_20180920_frames000204.jpg' '7' 1]
['20180920' '7_20180920_frames000205.jpg' '7' 1]
['20180920' '7_20180920_frames000206.jpg' '7' 1]
['20180920' '7_20180920_frames000207.jpg' '7' 1]
['20180920' '7_20180920_frames000208.jpg' '7' 1]
['20180920' '7_20180920_frames000209.jpg' '7' 1]
['20180920' '7_20180920_frames000210.jpg' '7' 1]
['20180920' '7_20180920_frames000211.jpg' '7' 1]
['20180920' '7_20180920_frames000212.jpg' '7' 1]
['20180920' '7_20180920_frames000213.jpg' '7' 1]
['20180920' '7_20180920_frames000214.jpg' '7' 1]
['20180920' '7_20180

['20180920' '7_20180920_frames000366.jpg' '7' 0]
['20180920' '7_20180920_frames000367.jpg' '7' 0]
['20180920' '7_20180920_frames000368.jpg' '7' 0]
['20180920' '7_20180920_frames000369.jpg' '7' 0]
['20180920' '7_20180920_frames000370.jpg' '7' 0]
['20180920' '7_20180920_frames000371.jpg' '7' 0]
['20180920' '7_20180920_frames000372.jpg' '7' 0]
['20180920' '7_20180920_frames000373.jpg' '7' 0]
['20180920' '7_20180920_frames000374.jpg' '7' 0]
['20180920' '7_20180920_frames000375.jpg' '7' 0]
['20180920' '7_20180920_frames000376.jpg' '7' 0]
['20180920' '7_20180920_frames000377.jpg' '7' 0]
['20180920' '7_20180920_frames000378.jpg' '7' 0]
['20180920' '7_20180920_frames000379.jpg' '7' 0]
['20180920' '7_20180920_frames000380.jpg' '7' 0]
['20180920' '7_20180920_frames000381.jpg' '7' 0]
['20180920' '7_20180920_frames000382.jpg' '7' 0]
['20180920' '7_20180920_frames000383.jpg' '7' 0]
['20180920' '7_20180920_frames000384.jpg' '7' 0]
['20180920' '7_20180920_frames000385.jpg' '7' 0]
['20180920' '7_20180

['20180920' '7_20180920_frames000541.jpg' '7' 0]
['20180920' '7_20180920_frames000542.jpg' '7' 0]
['20180920' '7_20180920_frames000543.jpg' '7' 0]
['20180920' '7_20180920_frames000544.jpg' '7' 0]
['20180920' '7_20180920_frames000545.jpg' '7' 0]
['20180920' '7_20180920_frames000546.jpg' '7' 0]
['20180920' '7_20180920_frames000547.jpg' '7' 0]
['20180920' '7_20180920_frames000548.jpg' '7' 0]
['20180920' '7_20180920_frames000549.jpg' '7' 0]
['20180920' '7_20180920_frames000550.jpg' '7' 0]
['20180920' '7_20180920_frames000551.jpg' '7' 0]
['20180920' '7_20180920_frames000552.jpg' '7' 0]
['20180920' '7_20180920_frames000553.jpg' '7' 0]
['20180920' '7_20180920_frames000554.jpg' '7' 0]
['20180920' '7_20180920_frames000555.jpg' '7' 0]
['20180920' '7_20180920_frames000556.jpg' '7' 0]
['20180920' '7_20180920_frames000557.jpg' '7' 0]
['20180920' '7_20180920_frames000558.jpg' '7' 0]
['20180920' '7_20180920_frames000559.jpg' '7' 0]
['20180920' '7_20180920_frames000560.jpg' '7' 0]
['20180920' '7_20180

In [36]:
labeling = pd.DataFrame(hh_dict, index =range(len(hh_dict['img_name'])))

In [37]:
pd.DataFrame.to_csv(labeling, label_path, index=False)